In [1]:
#El anterior día definimos variables de entorno. Ahora:

import os #Nos permite interactuar con el OS. 
os.environ["SPARK_HOME"]="/home/software/spark-3.1.2-bin-hadoop3.2/"
os.environ["HADOOP_CONF_DIR"]="/usr/local/hadoop/etc/hadoop" #Esto es para utilizar los ficheros de hadoop
os.environ["PYSPARK_PYTHON"]="python3.9" #Cualquier cosa que ejecutemos en python se va a ejecutar con esa versión
os.environ["PYSPARK_DRIVER_PYTHON"]="python3.9" #El driver es un orquestrador que tiene n nodos en el clúster. en un momento dado podríamos tener diferentes versiones entre el nodo y los drivers. Aquí estamos marcando la versión del lenguaje para todos. 


In [2]:
#pip install findspark

In [3]:
import findspark # Nos aseegura que la ruta que está definida  spark, para acceder y comunicarnos con ella. 

In [4]:
findspark.init() #define las variables de entorno

In [5]:
#pip install pyspark

In [6]:
from pyspark.sql import SparkSession #iniciar sesión
from pyspark import SparkContext, SparkConf #permite ajustar los parámetros en la configuración, cada vez que iniciemos sesión y ejecutemos trabajos. 

In [8]:
conf = SparkConf().setAll([
    ('spark.app.name','Sistemas Distribuidos'), #este es el nombre de la aplicación
    ('spark.executor.memory','1g'),  #cada uno de los nodos del driver van a tener 1G de memoria.  
    ('spark.executor.cores','8'),  #cores por cada uno de los nodos ejecutores. 
    ('spark.ui.port','4050'),  #puerto que me permite ver los jobs que se están ejecutando y otras operaciones dentro Spark. 
    ('spark.driver.memory','1g')]) #memoria asociada al driver. 

spark = SparkSession.builder.config(conf=conf).master('yarn').getOrCreate()  #crea una sesión con la configuración definida arriba en un clúster yarn y si tengo una sesión corriendo, únete, y si no crea una sesión. 
sc =spark.sparkContext #defino una variable para invocar el contexto. Es diferente a configuración. 
sc.setLogLevel('ERROR') #Esta declaración es para reducir el número de mensajes de errores 
    

Importante, primero instalar findspark y pyspark y luego correr el resto de código. 

green2020=spark.read.csv('hdfs://tucan:9000/NYCOpenData/2020/2020_Green_Taxi_Trip_Data.csv',header=True,inferSchema=True) # Accedemos al clúster HDFS y obtenemos el fichero Green Taxi Tripo Data

In [13]:
green2020.count()

1734051

In [15]:
green2020.printSchema(), # el nullable = true indica que ha dejado pasar datos nulos.

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



(None,)

Cuando le indicamos en el método .csv() inferSchema=True, le ordenamos a Spark que identifique el tipo de dato y lo ha hecho bien. 

In [19]:
green2020.show(10,False) #muestra las columnas. el primer parámetro indica el número de filas y el segundo parámetro cuántos caracteres quiero que aparezcan. Si quiero que no me resuma los caracteres, escribo False. 

+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime  |lpep_dropoff_datetime |store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|2       |12/18/2019 03:52:30 PM|12/18/2019 03:54:39 PM|N                 |1         |264         |264         |5              |0.0          |3.5        |

In [24]:
green2020.filter(green2020.passenger_count<1).count() #Estos datos no parecen ser relevantes. Por qué tenemos cuentas sin pasajeros?

3198

In [21]:
green2020.filter(green2020.fare_amount>1).show(10,False)

+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime  |lpep_dropoff_datetime |store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|2       |12/18/2019 03:52:30 PM|12/18/2019 03:54:39 PM|N                 |1         |264         |264         |5              |0.0          |3.5        |

In [26]:
green2020.filter(green2020.trip_distance==0.0).show(10,False) #este es otro ejemplo de datos atípicos que habría entender por qué están ahí y decidir si los debemos eliminar

+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime  |lpep_dropoff_datetime |store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|2       |12/18/2019 03:52:30 PM|12/18/2019 03:54:39 PM|N                 |1         |264         |264         |5              |0.0          |3.5        |

In [36]:
from pyspark.sql.functions import col, round # A diferencia de Pandas, en Spark debemos importar columnas y decirle todo el rato a Spark que una columa, es una columna
green2020 = green2020.withColumn('Tip_percentage',
                              round(100.0*(col('Tip_amount')/(col('Total_amount')-col('Tip_amount'))),3)) #el método withColumn permite añadir columna

In [37]:
green2020.show(5,False)

+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------+
|VendorID|lpep_pickup_datetime  |lpep_dropoff_datetime |store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|Tip_percentage|
+--------+----------------------+----------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------+
|2       |12/18/2019 03:52:30 PM|12/18/2019 03:54:39 PM|N                 |1         |264         |264       

En Spark puedo modificar variables, como lo que pasó cuando agregué una columna de "Tip Percentage". No obstante en Scala, los data Frames son inmutables. 

Pandas no es capaz de correr en más de 23 nodos, por eso se hace necesario Spark y Scala. 